<a href="https://colab.research.google.com/github/rdkworld/AIPND-2022/blob/main/Generalized/Train_an_Existing_Pytorch_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pre-requisite Setup

###Get Libraries

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 1.12.1+cu113
torchvision version: 0.13.1+cu113


###Regular Imports

In [3]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

#Additions from functions
import os

###Helpers/functions from Github

In [4]:
# Try to import the helper functions, download it from GitHub if it doesn't work
try:
    import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the scripts
    print("[INFO] Couldn't find the scripts... downloading them from GitHub.")
    !git clone https://github.com/rdkworld/AIPND-2022
    !mv AIPND-2022/Generalized/*.py .
    !rm -rf AIPND-2022
    import data_setup, engine, model_builder, utils 
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find the scripts... downloading them from GitHub.
Cloning into 'AIPND-2022'...
remote: Enumerating objects: 284, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 284 (delta 83), reused 116 (delta 53), pack-reused 110
Receiving objects: 100% (284/284), 11.00 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (105/105), done.


### Connect Colab and Google Drive to save and load models

In [ ]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

###Setup target device

In [7]:
# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## User input Parameters including Hyperparameters

In [15]:
#Data
SOURCE_URL = 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
BASE_DATA_DIRECTORY = 'data'
PROJECT_DATA_DIRECTORY = 'flowers'
FILE_NAME = 'flowers.tar.gz'
train_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/train"
valid_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/valid"
test_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/test"

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 64
HIDDEN_UNITS = '' #Not used
LEARNING_RATE = 0.003
MODEL_NAME = 'ViT_B_16'
LOSS_FUNCTION = 'CrossEntropyLoss'
OPTIMIZER = 'Adam'
MANUAL_RESIZE = 64


##Download data and categorize into train/valid/test folders as required

In [16]:
#Work in Progress, See next cell
!ls

data	       engine.py	    model_builder.py  __pycache__  train.py
data_setup.py  helper_functions.py  predictions.py    sample_data  utils.py


In [12]:
#Temporarily copy from Google Drive
#!mkdir $BASE_DATA_DIRECTORY
!cp /content/drive/MyDrive/flowers.tar.gz data/ #copy from drive to colab